0. filter subjects for those with more than 3 hits on every target
1. compute EMG ends -- the emg activity for hits, for each target
2. subtract this activity from the trial activity to get an "error" signal
3. for each target's activity, compute covariance of the error
4. project each of those covariances onto task and null space

In [1]:
import sys
if not '..' in sys.path:
    sys.path.append('..')
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colormaps
from analysis import collection, analysis, emg
from analysis import plot as analysis_plot
import scipy as sp
import pandas as pd
from sklearn.mixture import BayesianGaussianMixture
from sklearn.base import clone
import seaborn as sns
import pickle
from scipy.stats import multivariate_normal

In [2]:
collection_name = "emg_olympics"
task_name = "center_hold"
c = collection.Collection(collection_name)
subjects = c.alphabetically_sorted_subjects()

In [ ]:
NUM_ENDPOINTS = 5
MIN_HITS = 3

# get emg ending activity means over hit trials for each target
hit_ends = {}
for subject in c.alphabetically_sorted_subjects():
    hit_ends[subject.name] = []
    # targets
    for target_idx in range(1,13):
        target_hit_ends = []
        # trials
        for session in list(subject.tasks[task_name].sorted_sessions()):
            for trial in session.sorted_trials():
                # only hits on that target
                if trial.target_number == target_idx:
                    if trial.outcome == "Hit":
                        target_hit_ends.append(np.mean(trial.get_filtered_emg()[-NUM_ENDPOINTS:,:], axis=0))
        # all the end activity for this target's hits
        # this might be empty if there are no hits on that target in the whole session
        # in this case, we should put None and deal with it later
        if len(target_hit_ends) > MIN_HITS:
            hit_ends[subject.name].append(np.array(target_hit_ends))
        else:
            hit_ends[subject.name].append(None)